In [1]:
import tulip as tlp
from tulip.interfaces import omega as omega_int
from tulip import transys, abstract, spec, synth
from visualization import graph_builder as gb
import networkx as nx
from tulip.transys import machines
from GR1_defaults import settings

from tulip import dumpsmach
import pickle

`omega.symbolic.symbolic` failed to import `dd.cudd`.
Will use `dd.autoref`.
/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/tulip-1.4.0-py3.8.egg/tulip/transys/mathset.py:719: SyntaxWarning: "is" with a literal. Did you mean "=="?
/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/tulip-1.4.0-py3.8.egg/tulip/transys/mathset.py:719: SyntaxWarning: "is" with a literal. Did you mean "=="?
/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/tulip-1.4.0-py3.8.egg/tulip/transys/labeled_graphs.py:1218: SyntaxWarning: "is" with a literal. Did you mean "=="?
/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/tulip-1.4.0-py3.8.egg/tulip/transys/labeled_graphs.py:1226: SyntaxWarning: "is" with a literal. Did you mean "=="?
/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/tulip-1.4.0-py3.8.egg/tulip/transys/labeled_graphs.py:1218: SyntaxWarning: "is" with a literal

In [2]:
path = 'patrolling_car/'

In [3]:
# Define the states of the system
states = []
for x in range(5):
    for y in range(5):
        states.append("c{}{}".format(x, y))

In [4]:
# Variables
max_fuel = 14
env_vars = {'b': (0,4)}
sys_vars = {'r': states,
            'fuel': (-1,max_fuel),
            'move': 'boolean'
}

In [5]:
# Initialization
# env_init = {'b=2'}
env_init = {'b=0'}
sys_init = {'r="c40"',
            f'fuel={max_fuel}',
            '!move'
}

In [6]:
# Safety
# Blocker cannot self-loop (in patrolling_car_copy)
env_safe = {'(b=0) -> X(b=1)', 
            '(b=4) -> X(b=3)'}
for i in range(1,4):
    env_safe |= {'(b={0}) -> X((b={1}) | (b={2}))'.format(i, i-1, i+1)}

In [7]:
sys_safe = set()

for x in range(1,4):
    sys_safe |= {'(r="c{}{}") -> ((X(r="c{}{}") & (X !move)) | (X((r="c{}{}") | (r="c{}{}") | (r="c{}{}")) & X move))'.format(x,0, x,0, x-1,0, x+1,0, x,1),
                '(r="c{}{}") -> ((X(r="c{}{}") & (X !move)) | (X((r="c{}{}") | (r="c{}{}") | (r="c{}{}")) & X move))'.format(x,4,x,4, x-1,4, x+1,4, x,3),
                '(r="c{}{}") -> ((X(r="c{}{}") & (X !move)) | (X((r="c{}{}") | (r="c{}{}") | (r="c{}{}")) & X move))'.format(0,x,0,x, 0,x+1, 0,x-1, 1,x),
                '(r="c{}{}") -> ((X(r="c{}{}") & (X !move)) | (X((r="c{}{}") | (r="c{}{}") | (r="c{}{}")) & X move))'.format(4,x,4,x, 4,x+1, 4,x-1, 3,x)
    }
    for y in range(1,4):
        sys_safe |= {'(r="c{}{}") -> ((X(r="c{}{}") & !(X move)) | (X((r="c{}{}") | (r="c{}{}") | (r="c{}{}") | (r="c{}{}")) & X move))'.format(x,y,x,y, x+1,y, x-1,y, x,y+1, x,y-1)}

# Corners
sys_safe |= {'(r="c00") -> ((X(r="c00") & (X !move)) | (X((r="c10") | (r="c01")) & X move))',
            '(r="c40") -> ((X(r="c40") & (X !move)) | (X((r="c30") | (r="c41")) & X move))',
            '(r="c44") -> ((X(r="c44") & (X !move)) | (X((r="c34") | (r="c43")) & X move))',
            '(r="c04") -> ((X(r="c04") & (X !move)) | (X((r="c03") | (r="c14")) & X move))'
}

sys_safe |= {
            f'X(r="c42") -> X(fuel = {max_fuel})',
            'X(fuel > -1)',
            # Decreasing fuel when moving
            '((X move) && !X(r="c42")) -> (X(fuel) = fuel-2)',
            # Fuel decreasing when not moving
            '((X !move) && !X(r="c42") && !(r="c04")) -> (X(fuel) = fuel-1)',
            # No more losing fuel when at the goal
            '((X !move) && (r="c04")) -> (X(fuel) = fuel)'
    }
for i in range(0,5):
    sys_safe |= {'!((r="c1{0}") & (b={0}))'.format(i)}

In [8]:
# Progress
# env_prog = set()
env_prog = {'b=0'}
sys_prog = {'r="c04"'}

In [9]:
# Settings for the specifications
specs = settings.set_specs(env_vars, sys_vars, env_init, sys_init,
                            env_safe, sys_safe, env_prog, sys_prog)
print(specs.pretty())

ENVIRONMENT VARIABLES:
	b	(0, 4)

SYSTEM VARIABLES:
	r	['c00', 'c01', 'c02', 'c03', 'c04', 'c10', 'c11', 'c12', 'c13', 'c14', 'c20', 'c21', 'c22', 'c23', 'c24', 'c30', 'c31', 'c32', 'c33', 'c34', 'c40', 'c41', 'c42', 'c43', 'c44']
	fuel	(-1, 14)
	move	boolean

FORMULA:
ASSUMPTION:
    INITIAL
	  (b=0)
    SAFETY
	  []((b=4) -> X(b=3))
	& []((b=0) -> X(b=1))
	& []((b=2) -> X((b=1) | (b=3)))
	& []((b=3) -> X((b=2) | (b=4)))
	& []((b=1) -> X((b=0) | (b=2)))
    LIVENESS
	  []<>(b=0)
GUARANTEE:
    INITIAL
	  (r="c40")
	& (fuel=14)
	& (!move)
    SAFETY
	  []((r="c12") -> ((X(r="c12") & !(X move)) | (X((r="c22") | (r="c02") | (r="c13") | (r="c11")) & X move)))
	& []((r="c41") -> ((X(r="c41") & (X !move)) | (X((r="c42") | (r="c40") | (r="c31")) & X move)))
	& []((r="c14") -> ((X(r="c14") & (X !move)) | (X((r="c04") | (r="c24") | (r="c13")) & X move)))
	& []((r="c40") -> ((X(r="c40") & (X !move)) | (X((r="c30") | (r="c41")) & X move)))
	& []((r="c10") -> ((X(r="c10") & (X !move)) | (X((r="c0

In [10]:
# Turning the specifications into an automaton
# spec = tlp.synth._spec_plus_sys(specs, sys, None, False, False)
spec = tlp.synth._spec_plus_sys(specs, None, None, False, False)
aut = omega_int._grspec_to_automaton(spec)

In [11]:
# Synthesize the controller
# ctrl = tlp.synth.synthesize(specs, sys=sys)
ctrl = tlp.synth.synthesize(specs)
assert ctrl is not None, 'unrealizable'
with open(path + "ctrl", "wb") as file:
    pickle.dump(ctrl, file)

dumpsmach.write_python_case(path + 'controller.py', ctrl, classname="sys_ctrl")

removed 0 nodes from 66 total


In [12]:
# Graphing
filename = path + "graph"
attributes = ['color', 'shape']

In [13]:
# Making a graph of the asynchronous GR(1) game with deadends.
g0 = gb.game_graph(aut, env='env', sys='sys', remove_deadends=False, append_non_visited=True, qinit=aut.qinit)

about to create game graph
found winning set
before queue


In [14]:
with open(filename, "wb") as file:
    pickle.dump(g0, file)

In [15]:
h0 = gb._game_format_nx(g0, attributes)

In [16]:
pd0 = nx.drawing.nx_pydot.to_pydot(h0)

In [17]:
type(pd0)

pydot.Dot

In [18]:
len(pd0.get_edge_list())

6433

In [20]:
len(pd0.get_node_list())

2783

In [19]:
pd0.write_pdf(path + 'game.pdf')
print('wrote pd0')

wrote pd0
